In [36]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os
from tqdm.notebook import tqdm
from helper import check_path, seed_everything
from collections import defaultdict

In [37]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [39]:
EXPERIMENT_FAMILY_NAME = 'keras'
EXPERIMENT_NAME = 'labse_emb'
N_RANDOM_SEEDS = 1
N_EPOCHS = 100
BATCH_SIZE = 64

In [40]:
RANDOM_STATE = 77
seed_everything(RANDOM_STATE)

In [41]:
def filter_rare_categories(df: pd.DataFrame) -> pd.DataFrame:
    rare_categories = [12]
    return df.loc[~df[cfg.TARGET].isin(rare_categories)]

In [42]:

train = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'train.pkl'))
test = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'test.pkl'))

train = train.rename(columns={cfg.TEXT_COL: 'text'})
test = test.rename(columns={cfg.TEXT_COL: 'text'})

CLASSES = np.sort(train[cfg.TARGET].unique()).tolist()

# train = filter_rare_categories(train)

In [43]:
# EMB_NAME = 'smaller_LaBSE_15lang'
# train_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, EMB_NAME, 'train.pkl'))
# test_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, EMB_NAME, 'test.pkl'))

# train = train.join(train_emb)
# test = test.join(test_emb)

# del train_emb, test_emb; gc.collect()

In [44]:
X_train, y_train = train.drop(cfg.TARGET, axis=1), train[cfg.TARGET]
y_train_onecol = y_train.copy()

# y_train = pd.get_dummies(y_train)

N_OUT_CLASSES = len(CLASSES)
# assert y_train.shape[1] == N_OUT_CLASSES

In [45]:
theme_train = pd.read_csv(cfg.ORIG_TRAIN_PATH, index_col=cfg.ID_COL, usecols=[cfg.ID_COL, cfg.THEME_COL])
theme_train = theme_train[cfg.THEME_COL].astype('category').cat.codes
N_THEMES = theme_train.nunique()
# theme_train = pd.get_dummies(theme_train)
# theme_train.columns = [f'theme_{i}' for i in range(theme_train.shape[1])]


In [46]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(CLASSES))), index=train.index, columns=CLASSES)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(CLASSES))), index=test.index, columns=CLASSES)

In [47]:
from typing import Optional


def df_to_dataset(data: pd.DataFrame, labels: Optional[pd.DataFrame]=None, shuffle: bool=True, batch_size: int=32, labels2: Optional[pd.DataFrame]=None):
  df = data.copy()
  df = {key: value.values[:, tf.newaxis] for key, value in data.items()}
  
  if labels is None:
    ds = tf.data.Dataset.from_tensor_slices((dict(df),))
  else:
    ds = tf.data.Dataset.from_tensor_slices((
      dict(df), 
      {
        'category_out': labels, 
        'theme_out': labels2
        }
        ))
        
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds


def get_keras_input(
    data: pd.DataFrame,
    labels: Optional[pd.DataFrame] = None,
    labels2: Optional[pd.DataFrame] = None,
    shuffle=False
    ) -> tf.data.Dataset:
    return df_to_dataset(
        data=data,
        labels=labels,
        labels2=labels2,
        shuffle=shuffle, 
        batch_size=BATCH_SIZE)
        


In [48]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1", name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1", trainable=False, name='encoder')
    outputs = encoder(encoder_inputs)

    pooled_default = outputs['pooled_output']
    pooled_max = tf.keras.layers.GlobalMaxPool1D()(outputs['sequence_output'])
    net = tf.keras.layers.Concatenate()([pooled_default, pooled_max])
    
    net = tf.keras.layers.Dropout(0.3)(net)
    net = tf.keras.layers.Dense(300, activation=None)(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    category_out = tf.keras.layers.Dense(N_OUT_CLASSES, activation=None, name='category_out')(net)
    theme_out = tf.keras.layers.Dense(N_THEMES, activation=None, name='theme_out')(net)

    outputs = {
        'category_out': category_out, 
        'theme_out': theme_out
        }
    model = tf.keras.Model(
        text_input, 
        outputs=outputs
        )

    loss = {
        'category_out': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, name='category_out'),
        'theme_out': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, name='theme_out')
        }
    # metrics = tf.keras.metrics.AUC(from_logits=True, name='auc')

    epochs = N_EPOCHS
    n = (len(train) // cfg.N_SPLITS) * (cfg.N_SPLITS-1)
    steps_per_epoch = n // BATCH_SIZE
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-4
    optimizer = optimization.create_optimizer(
        init_lr=init_lr,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        optimizer_type='adamw')

    model.compile(
        optimizer=optimizer,
        loss=loss,
        loss_weights={"category_out": 1.0, "theme_out": 0.7}
        #  metrics=metrics
        )
    return model
    

In [49]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
leak_mask = leak_test.notnull()

In [50]:
cv = StratifiedKFold(n_splits=cfg.N_SPLITS, random_state=cfg.RANDOM_STATE, shuffle=True)

test_pool = get_keras_input(
        data=test,
        shuffle=False
        )


metrics = defaultdict(list)
fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, y_train_onecol), total=cfg.N_SPLITS):

    train_pool = get_keras_input(
        data=X_train.iloc[train_idx], 
        labels=y_train.iloc[train_idx],
        labels2=theme_train.iloc[train_idx],
        shuffle=True
        )

    val_pool = get_keras_input(
        data=X_train.iloc[val_idx], 
        labels=y_train.iloc[val_idx],
        labels2=theme_train.iloc[val_idx],
        shuffle=False
        )
        
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):

        clf = build_classifier_model()
        
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_category_out_loss', patience=3, min_delta=1e-5, mode='auto', restore_best_weights=True)
        clf.fit(x=train_pool, validation_data=val_pool, epochs=N_EPOCHS, callbacks=[early_stopping])
        
        model_name = f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}'
        model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
        check_path(model_path)
        clf.save(os.path.join(model_path, model_name), include_optimizer=False)
        
        pred_proba_oof_val = tf.nn.softmax(clf.predict(val_pool)['category_out']).numpy()
        pred_proba_oof_train = tf.nn.softmax(clf.predict(train_pool)['category_out']).numpy()
        pred_proba_oof.iloc[val_idx, :] += pred_proba_oof_val
        pred_proba_test.iloc[:, :] += tf.nn.softmax(clf.predict(test_pool)['category_out']).numpy()

        y_val_oof = y_train_onecol.iloc[val_idx]

        val_auc = roc_auc_score(y_val_oof, pred_proba_oof_val , multi_class='ovo', labels=CLASSES)
        metrics['val_auc'].append(val_auc)
        print('val auc', val_auc)

        leak_test_auc_score = roc_auc_score(leak_test.loc[leak_mask], pred_proba_test.loc[leak_mask] / (fold + 1), multi_class='ovo', labels=CLASSES)
        metrics['leak_test_auc_score'].append(leak_test_auc_score)
        print('leak_test_auc_score', leak_test_auc_score)
        
        del clf; gc.collect()
    del train_pool,val_pool; gc.collect() 
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (cfg.N_SPLITS * N_RANDOM_SEEDS)

  0%|          | 0/5 [00:00<?, ?it/s]

/home/as/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
25/25 [==============================] - 72s 2s/step - loss: 10.1469 - category_out_loss: 5.1577 - theme_out_loss: 7.1275 - val_loss: 6.6294 - val_category_out_loss: 2.4662 - val_theme_out_loss: 5.9474
Epoch 2/100
25/25 [==============================] - 39s 2s/step - loss: 7.2455 - category_out_loss: 2.7707 - theme_out_loss: 6.3927 - val_loss: 5.1886 - val_category_out_loss: 1.6584 - val_theme_out_loss: 5.0430
Epoch 3/100
25/25 [==============================] - 40s 2s/step - loss: 5.9471 - category_out_loss: 2.1280 - theme_out_loss: 5.4558 - val_loss: 4.4580 - val_category_out_loss: 1.2801 - val_theme_out_loss: 4.5400
Epoch 4/100
25/25 [==============================] - 46s 2s/step - loss: 5.2378 - category_out_loss: 1.7417 - theme_out_loss: 4.9945 - val_loss: 4.0300 - val_category_out_loss: 1.0662 - val_theme_out_loss: 4.2340
Epoch 5/100
25/25 [==============================] - 44s 2s/step - loss: 4.5941 - category_out_loss: 1.3837 - theme_out_loss: 4.5863 - val_loss: 3.

16/16 [==============================] - 20s 1s/step
val auc 0.881821955928904
leak_test_auc_score 0.8518888836723151


...

Epoch 1/100
25/25 [==============================] - 56s 2s/step - loss: 9.9875 - category_out_loss: 4.9517 - theme_out_loss: 7.1939 - val_loss: 6.3847 - val_category_out_loss: 2.2008 - val_theme_out_loss: 5.9770
Epoch 2/100
25/25 [==============================] - 47s 2s/step - loss: 7.2283 - category_out_loss: 2.6864 - theme_out_loss: 6.4885 - val_loss: 5.2726 - val_category_out_loss: 1.6665 - val_theme_out_loss: 5.1515
Epoch 3/100
25/25 [==============================] - 37s 1s/step - loss: 5.8883 - category_out_loss: 2.0092 - theme_out_loss: 5.5416 - val_loss: 4.4870 - val_category_out_loss: 1.3158 - val_theme_out_loss: 4.5303
Epoch 4/100
25/25 [==============================] - 48s 2s/step - loss: 5.0558 - category_out_loss: 1.5754 - theme_out_loss: 4.9720 - val_loss: 4.0199 - val_category_out_loss: 1.0947 - val_theme_out_loss: 4.1789
Epoch 5/100
25/25 [==============================] - 41s 2s/step - loss: 4.4945 - category_out_loss: 1.2975 - theme_out_loss: 4.5671 - val_loss: 3.6

16/16 [==============================] - 20s 1s/step
val auc 0.7512822392608872
leak_test_auc_score 0.8281817237845004


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
25/25 [==============================] - 59s 2s/step - loss: 8.4909 - category_out_loss: 3.5321 - theme_out_loss: 7.0840 - val_loss: 6.3618 - val_category_out_loss: 2.1517 - val_theme_out_loss: 6.0144
Epoch 2/100
25/25 [==============================] - 50s 2s/step - loss: 7.0187 - category_out_loss: 2.5505 - theme_out_loss: 6.3831 - val_loss: 5.2591 - val_category_out_loss: 1.6669 - val_theme_out_loss: 5.1316
Epoch 3/100
25/25 [==============================] - 39s 2s/step - loss: 6.0002 - category_out_loss: 2.1493 - theme_out_loss: 5.5012 - val_loss: 4.6144 - val_category_out_loss: 1.3553 - val_theme_out_loss: 4.6559
Epoch 4/100
25/25 [==============================] - 54s 2s/step - loss: 5.2318 - category_out_loss: 1.6922 - theme_out_loss: 5.0567 - val_loss: 4.1588 - val_category_out_loss: 1.1747 - val_theme_out_loss: 4.2629
Epoch 5/100
25/25 [==============================] - 40s 2s/step - loss: 4.6347 - category_out_loss: 1.4055 - theme_out_loss: 4.6131 - val_loss: 3.7

16/16 [==============================] - 20s 1s/step
val auc 0.8261490530331285
leak_test_auc_score 0.8392555674503177


In [51]:
oof_auc_score = roc_auc_score(y_train, pred_proba_oof , multi_class='ovo', labels=CLASSES)
print('oof_auc_score', oof_auc_score)
print('scores', metrics['val_auc'])
print('mean', np.mean(metrics['val_auc']), 'std', np.std(metrics['val_auc']))

oof_auc_score 0.798093907910142
scores [0.881821955928904, 0.9242831280465961, 0.8283456456500339, 0.7512822392608872, 0.8261490530331285]
mean 0.8423764043839099 std 0.05834894850777499


In [52]:
# 0.798093907910142

In [53]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
notnull = leak_test.notnull()
leak_test_auc_score = roc_auc_score(leak_test.loc[notnull], pred_proba_test.loc[notnull], multi_class='ovo', labels=CLASSES)
print('leak_test_auc_score', leak_test_auc_score)


leak_test_auc_score 0.8392555674503177


In [54]:
# 0.8392555674503177

In [55]:
submission = pd.read_csv(cfg.SAMPLE_SUBMIT_PATH).set_index('id')
assert submission.index.equals(pred_proba_test.index)
submission[cfg.TARGET] = pred_proba_test.idxmax(1)

submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))